<a href="https://colab.research.google.com/github/nitinpatil301a/Tensorflow-Keras-Work/blob/master/import_VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Using TensorFlow backend.


In [0]:
IMAGE_SIZE = [224, 224]
train_path = '/content/drive/My Drive/PetImages/train'
test_path = '/content/drive/My Drive/PetImages/test'

In [0]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [0]:
for layer in vgg.layers:
  layer.trainable = False

In [5]:
# useful for getting number of classes
folders = glob('/content/drive/My Drive/PetImages/train/*')
print(len(folders))

2


In [6]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [0]:
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [10]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = IMAGE_SIZE,
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = IMAGE_SIZE,
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 225 images belonging to 2 classes.
Found 52 images belonging to 2 classes.


In [0]:
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
checkpoint = ModelCheckpoint(filepath='mymodel.h5',verbose=1, save_best_only=True)
lr_reducer = ReduceLROnPlateau(factor=0.1,
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)
#callbacks = [checkpoint,lr_reducer]
callbacks = [checkpoint]

In [12]:
summary = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=callbacks
)

Epoch 1/10
8/8 [==============================] - 150s 19s/step - loss: 2.1374 - accuracy: 0.7867 - val_loss: 1.5331 - val_accuracy: 0.8654

Epoch 00001: val_loss improved from inf to 1.53310, saving model to mymodel.h5
Epoch 2/10
8/8 [==============================] - 140s 18s/step - loss: 1.1682 - accuracy: 0.9111 - val_loss: 1.5337 - val_accuracy: 0.9423

Epoch 00002: val_loss did not improve from 1.53310
Epoch 3/10
8/8 [==============================] - 140s 17s/step - loss: 0.8601 - accuracy: 0.9244 - val_loss: 1.2053 - val_accuracy: 0.8846

Epoch 00003: val_loss improved from 1.53310 to 1.20525, saving model to mymodel.h5
Epoch 4/10
8/8 [==============================] - 140s 18s/step - loss: 1.3033 - accuracy: 0.8800 - val_loss: 2.1905e-07 - val_accuracy: 0.9808

Epoch 00004: val_loss improved from 1.20525 to 0.00000, saving model to mymodel.h5
Epoch 5/10
8/8 [==============================] - 140s 17s/step - loss: 0.6036 - accuracy: 0.9422 - val_loss: 0.0000e+00 - val_accuracy: